In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import glob

client_credentials_manager = SpotifyClientCredentials(
        "ad3894b037f74b7390ab5bf093b438dc",
        "ccb7583fe8bd420886680798962e7ab2")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [2]:
def getTrackFeatures(trackURI):
    ger_data = pd.DataFrame()
    for track in trackURI:
        audio = sp.audio_analysis(track)
        segments_data =  pd.DataFrame(audio["segments"])
        pitch = segments_data.pitches.apply(pd.Series)
        pitch.columns = ["p" + str(i) for i in range(1,13)]
        timbre = segments_data.timbre.apply(pd.Series)
        timbre.columns = ["t" + str(i) for i in range(1,13)]
        segments_data = segments_data.drop(["pitches", "timbre", "loudness_end"],
                                           axis = 1)
        segments_data = segments_data.join([pitch, timbre])
        segments_data["track_id"] = track
        ger_data = ger_data.append(segments_data)
    return ger_data

In [ ]:
Africa = ['DZ', 'EG', 'MA', 'ZA', 'TN']
Asia = ['BH', 'HK', 'IN', 'ID', 'IL', 'JP', 'JO', 'KW', 
        'LB', 'MY', 'OM', 'PS', 'PH', 'QA', 'SA', 'SG', 
        'TW', 'TH', 'AE', 'VN']
Europe = ['AD', 'AT', 'BE', 'BG', 'CY', 'CZ', 'DK', 'EE', 
          'FI', 'FR', 'DE', 'GR', 'HU', 'IS', 'IE', 'IT', 
          'LV', 'LI', 'LT', 'LU', 'MT', 'MC', 'NL', 'NO', 
          'PL', 'PT', 'RO', 'SK', 'ES', 'SE', 'CH', 'TR', 'GB']
NorthAmerica = ['CA', 'CR', 'DO', 'SV', 'GT', 'HN', 'MX', 'NI', 'PA', 'US']
SouthAmerica = ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'PY', 'PE', 'UY']
Oceania = ['AU', 'NZ']

countriesOfInterest = Europe + NorthAmerica
#countriesOfInterest = ["HK", "JP", 'ZA', 'TN', 'TR', 'GB', 'MX', 'US', 'CO', 'EC', 'AU', 'NZ']

print(len(countriesOfInterest))
for year in range(2019,2000, -1):
    tmstamp = str(year) + "-06-15T12:00:00"
    for countryCode in countriesOfInterest:
        if len(glob.glob("Raw Playlist Data\\" + countryCode + "_" + str(year) + ".csv")) == 0:
            country = sp.featured_playlists(country = countryCode, limit = 50)
            plist = country["playlists"]["items"]

            plist = pd.DataFrame.from_dict(plist)
        
            print(countryCode, "has", len(plist), "playlists in ", str(year))
            plist.to_csv("Raw Playlist Data\\" + countryCode + "_" + str(year) + ".csv", index = False)
            for uri in plist["uri"]:
                currplist = sp.playlist(uri)
                trackURI = []
                for track in currplist["tracks"]["items"]:
                    try:
                        trackURI = trackURI + [track["track"]["uri"]]
                    except:
                        continue
                print("Getting track features for", uri, "in", countryCode)
                try:
                    trackData = getTrackFeatures(trackURI)
                    trackData.to_csv("Raw Track Data\\" + countryCode + "_" + str(year) + "_" + uri[17:] + ".csv", index = False)
                except:
                    continue
            

43
CH has 11 playlists in  2016
Getting track features for spotify:playlist:37i9dQZF1DX3crNbt46mRU in CH
Getting track features for spotify:playlist:37i9dQZF1DWXDvpUgU6QYl in CH
Getting track features for spotify:playlist:37i9dQZF1DWWmZlZqe008w in CH
Getting track features for spotify:playlist:37i9dQZF1DX8FmtpZigIQk in CH
Getting track features for spotify:playlist:37i9dQZF1DX05xCBTd43pw in CH
Getting track features for spotify:playlist:37i9dQZF1DXdPec7aLTmlC in CH
